In [1]:
import pandas as pd
import numpy as np

In [55]:
data_fao = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\pivotted_fao4.csv", low_memory=False)
country_codes = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\CSV датотеки\country_codes.csv")
data_icd = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\CSV датотеки\mergedicd.csv", low_memory=False)
data_icd.drop(columns='Unnamed: 0', inplace=True)

In [56]:
data_icd

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1125,NaN,NaN,1955,07A,A000,1,1,8,107822,...,2013.0,2727.0,4126.0,NaN,NaN,214.0,40543.0,NaN,NaN,NaN
1,1125,NaN,NaN,1955,07A,A000,2,1,8,105750,...,1745.0,3465.0,5963.0,NaN,NaN,82.0,39632.0,NaN,NaN,NaN
2,1125,NaN,NaN,1955,07A,A001,1,1,8,1302,...,8.0,6.0,2.0,NaN,NaN,1.0,6.0,NaN,NaN,NaN
3,1125,NaN,NaN,1955,07A,A001,2,1,8,739,...,2.0,1.0,2.0,NaN,NaN,0.0,4.0,NaN,NaN,NaN
4,1125,NaN,NaN,1955,07A,A002,1,1,8,69,...,0.0,0.0,0.0,NaN,NaN,0.0,7.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506979,4070,NaN,NaN,2017,103,Y86,1,2,1,39,...,2.0,4.0,4.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5506980,4070,NaN,NaN,2017,103,Y86,2,2,1,20,...,3.0,3.0,9.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5506981,4070,NaN,NaN,2017,103,Y87,1,2,1,2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5506982,4070,NaN,NaN,2017,103,Y87,2,2,1,2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Mapping ICD 9 to ICD 10

In [57]:
import re

# Ones that are not true are false
mapping = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\icd_9_btl_to_icd_9_detailed_codes.tsv", delimiter='\t')
mapping['isSummary'] = mapping['isSummary'].fillna(False)

# The ones that are NaN, are already mapped, so we will have the same value in both columns
mapping.loc[mapping['ICD-9 BTL'].isna(), 'ICD-9 BTL'] = mapping.loc[mapping['ICD-9 BTL'].isna()]['ICD-9 Detailed']
mapping['ICD-9 BTL'] = mapping['ICD-9 BTL'].map(lambda val: re.split(',|\\-|\\s|\\+', str(val))[0])

mapping

,ICD-9 BTL,ICD-9 Detailed,Cause,isSummary
0,B00,000,All causes,True
1,B01,001-009,Intestinal infectious Diseases,True
2,B010,001,Cholera,False
3,B011,002.0,Typhoid fever,False
4,B012,004,Shigellosis,False
...,...,...,...,...
487,238,238,NEOPLASM OF UNCERTAIN BEHAVIOUR OF OTHER SITES...,False
488,239,239,NEOPLASM OF UNSPECIFIED NATURE,False
489,415,"415+ (415.0, 417)",OTHER DISEASES OF PULMONARY CIRCULATION,False
490,416,416,CHRONIC PULMONARY HEART DISEASE,False


## Create the mapping dict

In [58]:
import re

icd9btl_to_detailed = dict()

for cause in data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C'])].Cause.unique():
    find_mapping = mapping.loc[mapping['ICD-9 BTL'] == cause]['ICD-9 Detailed'].values

    if find_mapping:
        detailed = find_mapping[0]
        detailed = re.split(',|\\-|\\s|\\+', str(detailed))[0]
        if detailed != 'nan':
            detailed = detailed.replace('.', '')
            icd9btl_to_detailed[cause] = detailed

icd9btl_to_detailed

C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


{'B00': '000',
 'B01': '001',
 'B010': '001',
 'B011': '0020',
 'B012': '004',
 'B013': '003',
 'B014': '006',
 'B015': '007',
 'B019': '009',
 'B02': '010',
 'B020': '011',
 'B022': '013',
 'B023': '014',
 'B024': '015',
 'B025': '016',
 'B029': '016',
 'B03': '020',
 'B030': '020',
 'B032': '030',
 'B033': '032',
 'B034': '033',
 'B035': '034',
 'B036': '036',
 'B037': '037',
 'B038': '038',
 'B04': '045',
 'B040': '045',
 'B041': '050',
 'B042': '055',
 'B044': '060',
 'B045': '062',
 'B046': '070',
 'B047': '071',
 'B05': '080',
 'B050': '080',
 'B051': '081',
 'B052': '084',
 'B054': '086',
 'B06': '090',
 'B060': '090',
 'B061': '098',
 'B069': '098',
 'B07': '100',
 'B072': '120',
 'B073': '122',
 'B074': '125',
 'B075': '126',
 'B076': '121',
 'B08': '140',
 'B09': '150',
 'B090': '150',
 'B091': '151',
 'B093': '153',
 'B094': '154',
 'B095': '1550',
 'B10': 'Malignant',
 'B100': '161',
 'B101': '162',
 'B109': '162',
 'B11': '170',
 'B110': '170',
 'B111': '172',
 'B113': '17

## Which BTL codes are summary

In [59]:
summary_btl_codes = set(mapping.loc[mapping['isSummary'] == True]['ICD-9 BTL'])

summary_btl_codes

{'B00',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',
 'B260',
 'B27',
 'B28',
 'B29',
 'B30',
 'B31',
 'B32',
 'B33',
 'B34',
 'B35',
 'B36',
 'B37',
 'B38',
 'B39',
 'B40',
 'B41',
 'B42',
 'B43',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B49',
 'B50',
 'B51',
 'B52',
 'B53',
 'B54',
 'B55',
 'B56',
 'C001',
 'C002',
 'C003',
 'C006',
 'C008',
 'C018',
 'C021',
 'C022',
 'C026',
 'C032',
 'C033',
 'C034',
 'C036',
 'C037',
 'C038',
 'C039',
 'C040',
 'C041',
 'C042',
 'C043',
 'C044',
 'C045',
 'C047',
 'C048',
 'C049',
 'C050',
 'C051',
 'C052',
 'C053',
 'C054',
 'C055',
 'C056',
 'C057',
 'C058',
 'C061',
 'C062',
 'C064',
 'C065',
 'C066',
 'C068',
 'C069',
 'C071',
 'C072',
 'C074',
 'C075',
 'C076',
 'C080',
 'C081',
 'C089',
 'C090',
 'C091',
 'C092',
 'C093',
 'C094',
 'C095',
 'C099',
 'C102',
 'C103',
 'CH17',
 'S47'}

## Set the isSummary column for all ICD 9 codes

In [60]:
data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']) & data_icd['Cause'].isin(summary_btl_codes), 'isSummary'] = True
data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']) & (~data_icd['Cause'].isin(summary_btl_codes)), 'isSummary'] = False

data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C'])]

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4,isSummary
281749,2005,NaN,NaN,1983,09A,B00,1,1,1,36,...,5.0,4.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,True
281750,2005,NaN,NaN,1983,09A,B00,2,1,1,37,...,5.0,10.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0,True
281751,2005,NaN,NaN,1983,09A,B01,1,1,1,1,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,True
281752,2005,NaN,NaN,1983,09A,B01,2,1,1,0,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,True
281753,2005,NaN,NaN,1983,09A,B010,1,1,1,0,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181214,3068,NaN,A51,2000,09C,C101,2,2,8,892,...,162.0,280.0,NaN,NaN,0.0,14.0,NaN,NaN,NaN,False
1181215,3068,NaN,A51,2000,09C,C102,1,2,8,2005,...,76.0,37.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,True
1181216,3068,NaN,A51,2000,09C,C102,2,2,8,1850,...,83.0,62.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,True
1181217,3068,NaN,A51,2000,09C,C103,1,2,8,957,...,8.0,4.0,NaN,NaN,0.0,3.0,NaN,NaN,NaN,True


## We do the mapping from BTL to detailed

In [61]:
data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']), 'Cause'] = \
    data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']), 'Cause'].map(icd9btl_to_detailed)

data_icd = data_icd.loc[~data_icd['Cause'].isna()]

data_icd

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4,isSummary
0,1125,NaN,NaN,1955,07A,A000,1,1,8,107822,...,2727.0,4126.0,NaN,NaN,214.0,40543.0,NaN,NaN,NaN,NaN
1,1125,NaN,NaN,1955,07A,A000,2,1,8,105750,...,3465.0,5963.0,NaN,NaN,82.0,39632.0,NaN,NaN,NaN,NaN
2,1125,NaN,NaN,1955,07A,A001,1,1,8,1302,...,6.0,2.0,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN
3,1125,NaN,NaN,1955,07A,A001,2,1,8,739,...,1.0,2.0,NaN,NaN,0.0,4.0,NaN,NaN,NaN,NaN
4,1125,NaN,NaN,1955,07A,A002,1,1,8,69,...,0.0,0.0,NaN,NaN,0.0,7.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506979,4070,NaN,NaN,2017,103,Y86,1,2,1,39,...,4.0,4.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506980,4070,NaN,NaN,2017,103,Y86,2,2,1,20,...,3.0,9.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506981,4070,NaN,NaN,2017,103,Y87,1,2,1,2,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506982,4070,NaN,NaN,2017,103,Y87,2,2,1,2,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN


## Now we map from ICD 9 detailed to ICD 10

### First table

In [62]:
icd9_to_icd10_table = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\icd9_to_icd10_mapping.csv")
icd_9_to_10_mapper = dict()

for index, row in icd9_to_icd10_table.iterrows():
    icd9 = row['icd9cm']
    icd10 = row['icd10cm']
    try:
        icd_9_int = int(icd9)
        icd_9_to_10_mapper[icd9] = icd10
        icd_9_to_10_mapper[str(icd_9_int)] = icd10
    except:
        icd_9_to_10_mapper[icd9] = icd10
        
icd_9_to_10_mapper

{'0010': 'A000',
 '10': 'A000',
 '0011': 'A001',
 '11': 'A001',
 '0019': 'A009',
 '19': 'A009',
 '0020': 'A0100',
 '20': 'A0100',
 '0021': 'A011',
 '21': 'A011',
 '0022': 'A012',
 '22': 'A012',
 '0023': 'A013',
 '23': 'A013',
 '0029': 'A014',
 '29': 'A014',
 '0030': 'A020',
 '30': 'A020',
 '0031': 'A021',
 '31': 'A021',
 '00320': 'A0220',
 '320': 'A360',
 '00321': 'A0221',
 '321': 'A361',
 '00322': 'A0222',
 '322': 'A3689',
 '00323': 'A0223',
 '323': 'A362',
 '00324': 'A0224',
 '324': 'A0224',
 '00329': 'A0229',
 '329': 'A369',
 '0038': 'A028',
 '38': 'A028',
 '0039': 'A029',
 '39': 'A029',
 '0040': 'A030',
 '40': 'A030',
 '0041': 'A031',
 '41': 'A031',
 '0042': 'A032',
 '42': 'B20',
 '0043': 'A033',
 '43': 'A033',
 '0048': 'A038',
 '48': 'A880',
 '0049': 'A039',
 '49': 'A039',
 '0050': 'A050',
 '50': 'A050',
 '0051': 'A051',
 '51': 'A051',
 '0052': 'A052',
 '52': 'A052',
 '0053': 'A058',
 '53': 'A058',
 '0054': 'A053',
 '54': 'A053',
 '00581': 'A055',
 '581': 'A055',
 '00589': 'A058',

### Second table

In [63]:
icd9_to_icd10_table2 = pd.read_excel(r"C:\Users\Aleksandar\Downloads\Mortality\icd9_to_icd10_mapping_2.xls")
icd_9_to_10_mapper2 = dict()

for index, row in icd9_to_icd10_table2.iterrows():
    icd9 = row['ICD9']
    icd10 = row['Pure Vic Logical']
    try:
        icd_9_int = int(icd9)
        icd_9_to_10_mapper2[icd9] = icd10
        icd_9_to_10_mapper2[str(icd_9_int)] = icd10
    except:
        icd_9_to_10_mapper2[icd9] = icd10
        
icd_9_to_10_mapper2

{'0010': 'A000',
 '10': 'A000',
 '0011': 'A001',
 '11': 'A001',
 '0019': 'A009',
 '19': 'A009',
 '0020': 'A010',
 '20': 'A010',
 '0021': 'A011',
 '21': 'A011',
 '0022': 'A012',
 '22': '4001200',
 '0023': 'A013',
 '23': 'A013',
 '0029': 'A014',
 '29': 'A014',
 '0030': 'A020',
 '30': 'A020',
 '0031': 'A021',
 '31': '4033000',
 '00320': 'A022',
 '320': 'A360',
 '00321': 'G01',
 '321': '9016500',
 '00322': 'J170',
 '322': 'A361',
 '00323': 'M0139',
 '323': '3843800',
 '00324': 'M9029',
 '324': '3843801',
 '00329': 'A022',
 '329': '3844000',
 '0038': 'A028',
 '38': '1823000',
 '0039': 'A029',
 '39': 'A029',
 '0040': 'A030',
 '40': 'A030',
 '0041': 'A031',
 '41': 'A031',
 '0042': 'A032',
 '42': '1829000',
 '0043': 'A033',
 '43': '3930000',
 '0048': 'A038',
 '48': 'A888',
 '0049': 'A039',
 '49': 'A039',
 '0050': 'A050',
 '50': '9001400',
 '0051': 'A051',
 '51': 'A051',
 '0052': 'A052',
 '52': 'A052',
 '0053': 'A058',
 '53': 'A058',
 '0054': 'A053',
 '54': 'A053',
 '00581': 'A058',
 '581': '37

In [64]:
icd_9_to_icd_10_combined_mapper = {**icd_9_to_10_mapper, **icd_9_to_10_mapper2}

### See which codes are not mapped

In [65]:
summary_codes_not_mapped = data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']) & \
                                        (~data_icd['Cause'].isin(icd_9_to_icd_10_combined_mapper.keys()))]['Cause'].unique()

summary_codes_not_mapped

array(['000', '001', '004', '003', '006', '007', '009', '010', '011',
       '013', '014', '015', '020', '030', '032', '033', '055', '060',
       '070', '081', '084', '086', '098', '120', '126', '140', '150',
       'Malignant', '162', '170', '174', '180', '188', '250', '264',
       '345', '410', '440', '451', '466', '487', '571', '574', '590',
       '635', '6424', '647', '710', '767', 'E800', 'E810', 'E850', 'E870',
       'E880', 'E890', 'E900', 'E910', 'E919', 'E922', 'E930', 'E950',
       'E960', 'E980', 'E990', 'E999', '023', '056', '076', '085', '137',
       '173', '183', '191', '216', '225', '2780', '4151', '454', '455',
       '464', '495', '564', '592', '606', '615', '618', '6466', '676',
       '7543', 'E826', 'E830', 'E840', 'E845', 'E860', '306', '143',
       '144', '145', '147', '148', '158', '160', '163', '164', '165',
       '175', '182', '187', '192', '195', '197', '198', '236', '237',
       '365', '366', '375', '378', '3720', '2000', '2002', '2030', '2040',
    

###  Map those codes to some detailed code (with a trailing zero or two zeros)

In [66]:
for code in summary_codes_not_mapped:
    if code + '0' in icd_9_to_icd_10_combined_mapper:
        icd_9_to_icd_10_combined_mapper[code] = icd_9_to_icd_10_combined_mapper[code + '0']
    elif code + '00' in icd_9_to_icd_10_combined_mapper:
        icd_9_to_icd_10_combined_mapper[code] = icd_9_to_icd_10_combined_mapper[code + '00']

## Map ICD 9 detailed to ICD 10

In [67]:
data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']), 'Cause'] = \
    data_icd.loc[data_icd['List'].isin(['09A', '09B', '09N', '09C']), 'Cause'].map(icd_9_to_icd_10_combined_mapper)
data_icd = data_icd.loc[~data_icd['Cause'].isna()]

data_icd

C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4,isSummary
0,1125,NaN,NaN,1955,07A,A000,1,1,8,107822,...,2727.0,4126.0,NaN,NaN,214.0,40543.0,NaN,NaN,NaN,NaN
1,1125,NaN,NaN,1955,07A,A000,2,1,8,105750,...,3465.0,5963.0,NaN,NaN,82.0,39632.0,NaN,NaN,NaN,NaN
2,1125,NaN,NaN,1955,07A,A001,1,1,8,1302,...,6.0,2.0,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN
3,1125,NaN,NaN,1955,07A,A001,2,1,8,739,...,1.0,2.0,NaN,NaN,0.0,4.0,NaN,NaN,NaN,NaN
4,1125,NaN,NaN,1955,07A,A002,1,1,8,69,...,0.0,0.0,NaN,NaN,0.0,7.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506979,4070,NaN,NaN,2017,103,Y86,1,2,1,39,...,4.0,4.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506980,4070,NaN,NaN,2017,103,Y86,2,2,1,20,...,3.0,9.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506981,4070,NaN,NaN,2017,103,Y87,1,2,1,2,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
5506982,4070,NaN,NaN,2017,103,Y87,2,2,1,2,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN


## Remove all columns that dont have food supply in them 

In [68]:
def filter_columns_by_name(data, include_string='', exclude_string='', excluded_columns=[]):
    columns = data.columns.values
    
    if include_string != '' and exclude_string != '':
        filtered_columns = [col for col in columns if include_string in col and exclude_string not in col]
    elif include_string == '' and exclude_string != '':
        filtered_columns = [col for col in columns if exclude_string not in col]
    elif include_string != '' and exclude_string == '':
        filtered_columns = [col for col in columns if include_string in col]
        
    filtered_columns.extend(excluded_columns)
    
    return data[filtered_columns]

In [69]:
data_fao = filter_columns_by_name(data_fao,\
                                  include_string='Food supply',\
                                  exclude_string='quantity',\
                                  excluded_columns=['Area Code', 'Year'])

data_fao

,"Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day","Animal Products,Food supply (kcal/capita/day),kcal/capita/day","Animal fats,Food supply (kcal/capita/day),kcal/capita/day","Apples and products,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Animals, Others,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Plants,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Products, Other,Food supply (kcal/capita/day),kcal/capita/day","Bananas,Food supply (kcal/capita/day),kcal/capita/day","Barley and products,Food supply (kcal/capita/day),kcal/capita/day","Beans,Food supply (kcal/capita/day),kcal/capita/day",...,"Treenuts,Food supply (kcal/capita/day),kcal/capita/day","Vegetable Oils,Food supply (kcal/capita/day),kcal/capita/day","Vegetables, Other,Food supply (kcal/capita/day),kcal/capita/day","Vegetables,Food supply (kcal/capita/day),kcal/capita/day","Vegetal Products,Food supply (kcal/capita/day),kcal/capita/day","Wheat and products,Food supply (kcal/capita/day),kcal/capita/day","Wine,Food supply (kcal/capita/day),kcal/capita/day","Yams,Food supply (kcal/capita/day),kcal/capita/day",Area Code,Year
0,32.0,355.0,29.0,23.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,11.0,39.0,75.0,1829.0,1215.0,14.0,NaN,1,1992
1,34.0,267.0,56.0,8.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,11.0,33.0,59.0,1832.0,1256.0,16.0,NaN,1,1993
2,58.0,372.0,56.0,12.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,60.0,39.0,76.0,1809.0,1127.0,14.0,NaN,1,1994
3,41.0,327.0,73.0,16.0,0.0,0.0,0.0,0.0,2.0,NaN,...,0.0,109.0,47.0,83.0,1943.0,1200.0,6.0,NaN,1,1995
4,41.0,312.0,60.0,15.0,0.0,0.0,0.0,0.0,3.0,NaN,...,0.0,129.0,53.0,85.0,2008.0,1117.0,5.0,NaN,1,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11256,24.0,273.0,34.0,3.0,0.0,0.0,0.0,20.0,19.0,33.0,...,7.0,191.0,25.0,40.0,2239.0,440.0,1.0,19.0,5817,2013
11257,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,/,/,5817,2014
11258,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,/,/,5817,2015
11259,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,/,/,5817,2016


In [70]:
data_fao_with_codes = data_fao.join(country_codes.set_index('FAO code'), on='Area Code')

data_fao_with_codes

,"Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day","Animal Products,Food supply (kcal/capita/day),kcal/capita/day","Animal fats,Food supply (kcal/capita/day),kcal/capita/day","Apples and products,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Animals, Others,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Plants,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Products, Other,Food supply (kcal/capita/day),kcal/capita/day","Bananas,Food supply (kcal/capita/day),kcal/capita/day","Barley and products,Food supply (kcal/capita/day),kcal/capita/day","Beans,Food supply (kcal/capita/day),kcal/capita/day",...,"Vegetables, Other,Food supply (kcal/capita/day),kcal/capita/day","Vegetables,Food supply (kcal/capita/day),kcal/capita/day","Vegetal Products,Food supply (kcal/capita/day),kcal/capita/day","Wheat and products,Food supply (kcal/capita/day),kcal/capita/day","Wine,Food supply (kcal/capita/day),kcal/capita/day","Yams,Food supply (kcal/capita/day),kcal/capita/day",Area Code,Year,Country,WHO code
0,32.0,355.0,29.0,23.0,0.0,0.0,0.0,0.0,0.0,NaN,...,39.0,75.0,1829.0,1215.0,14.0,NaN,1,1992,Armenia,4007.0
1,34.0,267.0,56.0,8.0,0.0,0.0,0.0,0.0,0.0,NaN,...,33.0,59.0,1832.0,1256.0,16.0,NaN,1,1993,Armenia,4007.0
2,58.0,372.0,56.0,12.0,0.0,0.0,0.0,0.0,0.0,NaN,...,39.0,76.0,1809.0,1127.0,14.0,NaN,1,1994,Armenia,4007.0
3,41.0,327.0,73.0,16.0,0.0,0.0,0.0,0.0,2.0,NaN,...,47.0,83.0,1943.0,1200.0,6.0,NaN,1,1995,Armenia,4007.0
4,41.0,312.0,60.0,15.0,0.0,0.0,0.0,0.0,3.0,NaN,...,53.0,85.0,2008.0,1117.0,5.0,NaN,1,1996,Armenia,4007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11256,24.0,273.0,34.0,3.0,0.0,0.0,0.0,20.0,19.0,33.0,...,25.0,40.0,2239.0,440.0,1.0,19.0,5817,2013,NaN,NaN
11257,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2014,NaN,NaN
11258,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2015,NaN,NaN
11259,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2016,NaN,NaN


In [71]:
data_fao_with_codes.loc[data_fao_with_codes['WHO code'].isna()]

,"Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day","Animal Products,Food supply (kcal/capita/day),kcal/capita/day","Animal fats,Food supply (kcal/capita/day),kcal/capita/day","Apples and products,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Animals, Others,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Plants,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Products, Other,Food supply (kcal/capita/day),kcal/capita/day","Bananas,Food supply (kcal/capita/day),kcal/capita/day","Barley and products,Food supply (kcal/capita/day),kcal/capita/day","Beans,Food supply (kcal/capita/day),kcal/capita/day",...,"Vegetables, Other,Food supply (kcal/capita/day),kcal/capita/day","Vegetables,Food supply (kcal/capita/day),kcal/capita/day","Vegetal Products,Food supply (kcal/capita/day),kcal/capita/day","Wheat and products,Food supply (kcal/capita/day),kcal/capita/day","Wine,Food supply (kcal/capita/day),kcal/capita/day","Yams,Food supply (kcal/capita/day),kcal/capita/day",Area Code,Year,Country,WHO code
596,184.0,951.0,364.0,21.0,0.0,0.0,0.0,11.0,1.0,5.0,...,51.0,58.0,1972.0,770.0,16.0,NaN,15,1961,NaN,NaN
597,181.0,958.0,391.0,22.0,0.0,0.0,0.0,11.0,1.0,9.0,...,52.0,58.0,1950.0,770.0,15.0,NaN,15,1962,NaN,NaN
598,190.0,958.0,386.0,19.0,0.0,0.0,0.0,10.0,1.0,10.0,...,56.0,63.0,2049.0,768.0,17.0,NaN,15,1963,NaN,NaN
599,198.0,939.0,366.0,22.0,0.0,0.0,0.0,11.0,1.0,6.0,...,59.0,66.0,2122.0,793.0,18.0,NaN,15,1964,NaN,NaN
600,198.0,936.0,366.0,24.0,0.0,0.0,0.0,11.0,1.0,11.0,...,49.0,56.0,2030.0,777.0,20.0,NaN,15,1965,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11256,24.0,273.0,34.0,3.0,0.0,0.0,0.0,20.0,19.0,33.0,...,25.0,40.0,2239.0,440.0,1.0,19.0,5817,2013,NaN,NaN
11257,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2014,NaN,NaN
11258,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2015,NaN,NaN
11259,/,/,/,/,/,/,/,/,/,/,...,/,/,/,/,/,/,5817,2016,NaN,NaN


## Function for filtering the entrire ICD dataset by cause (disease)

In [72]:
def filter_icd_by_cause(data, cause):
    filtered_data = data.loc[data_icd['Cause'].isin(cause)]
    return filtered_data

## Function for merging 2 datasets on a given column or several columns## 

In [73]:
def merge_datasets(data1, index1, data2, index2):
    merged = data1.join(data2.set_index(index2), on=index1, how='inner')
    return merged

## Filtering by a cause of death and merging the datasets## 

In [74]:
def filter_and_merge(data1, index1, data2, index2, cause):
    filtered_icd = filter_icd_by_cause(data2, cause)
    filtered_icd = filtered_icd.rename(columns={'Deaths1': 'Total Deaths'})
    
    total_deaths_agg = pd.DataFrame(filtered_icd.groupby(['Country', 'Year'])['Total Deaths'].agg('sum'))
    
    merged = merge_datasets(data1, index1, total_deaths_agg.reset_index(), index2)
    filtered_columns = ["WHO code"] #"Admin1", "Cause", "Frmat", "IM_Frmat", "List", "SubDiv"]
    merged = merged.drop(columns=filtered_columns)
    
    total_deaths_agg = total_deaths_agg.reset_index()
    merged = merged.set_index(['Country', 'Year'])
    
    return merged

In [75]:
#https://icd.who.int/browse10/2019/en#/I

diseaseCodesByGroup = dict()

# 1 certain_infectious_and_parasitic_diseases (A00-B99)
certain_infectious_and_parasitic_diseases = ["A{:02d}".format(val) for val in ( list(range(10)) + list(range(15, 29)) + \
                                                                              list(range(30, 90)) + list(range(92, 100)) ) ]
certain_infectious_and_parasitic_diseases.extend(["B{:02d}".format(val) for val in ( list(range(10)) + list(range(15, 84)) + \
                                                                                   list(range(85, 100)) ) ] )
diseaseCodesByGroup['certain_infectious_and_parasitic_diseases'] = certain_infectious_and_parasitic_diseases

# 2 Neoplasms (C00-D48)
neoplasms = ["C{:02d}".format(val) for val in range(98)]
neoplasms.extend(["D{:02d}".format(val) for val in range(49)])
diseaseCodesByGroup['neoplasms'] = neoplasms

# 3 Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism (D50-D89)
blood_immune_diseases = ["D{:02d}".format(val) for val in range(50, 90)]
diseaseCodesByGroup['blood_immune_diseases'] = blood_immune_diseases

# 4 Endocrine, nutritional and metabolic diseases (E00-E90)
endocrine_nutritional_metabolic_diseases = ["E{:02d}".format(val) for val in range(91)]
diseaseCodesByGroup['endocrine_nutritional_metabolic_diseases'] = endocrine_nutritional_metabolic_diseases

#5 Mental and behavioural disorders (F00-F99)
mental_behavioural_disorders = ["F{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['mental_behavioural_disorders'] = mental_behavioural_disorders

# 6 Diseases of the nervous system (G00-G99)
nervous_system_diseases = ["G{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['nervous_system_diseases'] = nervous_system_diseases

# 7 Diseases of the eye and adnexa (H00-H59)
eye_adnexa_diseases = ["H{:02d}".format(val) for val in range(60)]
diseaseCodesByGroup['eye_adnexa_diseases'] = eye_adnexa_diseases

# 8 Diseases of the ear and mastoid process (H60-H95)
ear_mastoid_process_diseases = ["H{:02d}".format(val) for val in range(60, 96)]
diseaseCodesByGroup['ear_mastoid_process_diseases'] = ear_mastoid_process_diseases

# 9 Diseases of the circulatory system (I00-I99)
circulatory_diseases = ["I{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['circulatory_diseases'] = circulatory_diseases

# 10 Diseases of the respiratory system (J00-J99)
respiratory_diseases = ["J{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['respiratory_diseases'] = respiratory_diseases

# 11 Diseases of the digestive system (K00-K93)
digestive_diseases = ["K{:02d}".format(val) for val in range(94)]
diseaseCodesByGroup['digestive_diseases'] = digestive_diseases

# 12 Diseases of the skin and subcutaneous tissue (L00-L99)
skin_subcutaneous_diseases = ["L{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['skin_subcutaneous_diseases'] = skin_subcutaneous_diseases

# 13 Diseases of the musculoskeletal system and connective tissue (M00-M99)
musculoskeletal_connective_diseases = ["M{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['musculoskeletal_connective_diseases'] = musculoskeletal_connective_diseases

# 14 Diseases of the genitourinary system (N00-N99)
genitourinary_diseases = ["N{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['genitourinary_diseases'] = genitourinary_diseases

# 15 Pregnancy, childbirth and the puerperium (O00-O99)
pregnancy_childbirth_puerperium = ["O{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['pregnancy_childbirth_puerperium'] = pregnancy_childbirth_puerperium

# 16 Certain conditions originating in the perinatal period (P00-P96)
perinatal_conditions = ["P{:02d}".format(val) for val in range(97)]
diseaseCodesByGroup['perinatal_conditions'] = perinatal_conditions

# 17 Congenital malformations, deformations and chromosomal abnormalities (Q00-Q99)
congenital_malformations_deformations_chromosomal_abnormalities = ["Q{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['congenital_malformations_deformations_chromosomal_abnormalities'] = congenital_malformations_deformations_chromosomal_abnormalities

# 18 Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)
symptoms_signs_findings_not_classified = ["R{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['symptoms_signs_findings_not_classified'] = symptoms_signs_findings_not_classified

# 19 Injury, poisoning and certain other consequences of external causes (S00-T98)
injury_poisoning_external_causes = ["S{:02d}".format(val) for val in range(100)]
injury_poisoning_external_causes.extend(["T{:02d}".format(val) for val in range(99)])
diseaseCodesByGroup['injury_poisoning_external_causes'] = injury_poisoning_external_causes

# 20 External causes of morbidity and mortality (V01-Y98)
external_causes = ["V{:02d}".format(val) for val in range(1, 100)]
external_causes.extend(["W{:02d}".format(val) for val in range(100)])
external_causes.extend(["X{:02d}".format(val) for val in range(100)])
external_causes.extend(["Y{:02d}".format(val) for val in range(100)])
diseaseCodesByGroup['external_causes'] = external_causes

# 21 Factors influencing health status and contact with health services(Z00-Z99)
factors_influencing_health_status_contact_with_health_services = ["Z{:02d}".format(val) for val in range(100)]
diseaseCodesByGroup['factors_influencing_health_status_contact_with_health_services'] = factors_influencing_health_status_contact_with_health_services

#Grouped more granular
diseaseCodesByGroup['malig_neoplasm_of_breast'] = ['C50']

diseaseCodesByGroup['malignant_neoplasms_of_digestive_organs'] = [f"C{val}" for val in range(15, 27)]

diseaseCodesByGroup['malignant_neoplasms_of_eye_brain_and_other_parts_of_central_nervous_system'] = \
    [f"C{val}" for val in range(69, 73)]

diseaseCodesByGroup['other_diseases_of_intestines'] = [f"K{val}" for val in range(55, 65)]

diseaseCodesByGroup['malignant_neoplasms_of_mesothelial_and_soft_tissue'] = [f"C{val}" for val in range(45, 50)]

diseaseCodesByGroup['malignant_neoplasms_of_urinary_tract'] = [f"C{val}" for val in range(64, 69)]

diseaseCodesByGroup['arthropod_borne_viral_fevers_and_viral_haemorrhagic_fevers'] = [f"A{val}" for val in range(92, 100)]

diseaseCodesByGroup['mycoses'] = [f"B{val}" for val in range(35, 49)]

diseaseCodesByGroup['diseases_of_peritoneum'] = [f"K{val}" for val in range(65, 68)]

diseaseCodesByGroup['cerebrovascular_diseases'] = [f"I{val}" for val in range(60, 70)]

diseaseCodesByGroup['ischaemic_heart_diseases'] = [f"I{val}" for val in range(20, 26)]

diseaseCodesByGroup['melanoma_and_other_malignant_neoplasms_of_skin'] = ["C43", "C44"]

diseaseCodesByGroup['demyelinating_diseases_of_the_central_nervous_system'] = ["G35", "G36", "G37"]

diseaseCodesByGroup['diabetes_mellitus'] = ["E10", "E11", "E12", "E13", "E14"]

diseaseCodesByGroup['diseases_of_liver'] = [f"K{val}" for val in range(70, 78)]

diseaseCodesByGroup['malig_neoplasm_of_female_genital_organs'] = [f"C{val}" for val in range(51, 59)]

diseaseCodesByGroup['malig_neoplasm_of_lip_oral_cavity_and_pharynx'] = ["C{:02d}".format(val) for val in range(0, 15)]

diseaseCodesByGroup['neoplasms_of_lymphoid_haematopoietic_and_related_tissue'] = [f"C{val}" for val in range(81, 97)]

diseaseCodesByGroup['malignant_neoplasm_of_male_genital_organs'] = ["C60", "C61", "C62", "C63"]

diseaseCodesByGroup['malig_neoplasms_of_respiratory_and_intrathoracic_organs'] = [f"C{val}" for val in range(30, 40)]

all_causes = [val for key, value in diseaseCodesByGroup.items() for val in value]
diseaseCodesByGroup['all_causes'] = ['B00']

diseaseCodesByGroup['non_circulatory_diseases'] = [val for val in all_causes \
                                                   if val not in diseaseCodesByGroup['circulatory_diseases']]

diseaseCodesByGroup['diabetes'] = [f"E{val}" for val in range(10, 15)]

diseaseCodesByGroup['myocardial_infarction'] = ["I21", "I22", "I23"]

diseaseCodesByGroup['liver_cirrhosis'] = ["K74"]

diseaseCodesByGroup['alcoholic_cirrhosis'] = ["K70"]

diseaseCodesByGroup['nonalcoholic_fatty_liver_disease'] = ["K76"]

diseaseCodesByGroup['nonalcoholic_steatohepatitis'] = ["K75"]

diseaseCodesByGroup['hepatocellular_carcinoma'] = ["C22"]

diseaseCodesByGroup['type_2_diabetes'] = ["E11"]

diseaseCodesByGroup['malignant_neoplasms_of_digestive_organs_and_malig_neoplasm_of_lip_oral_cavity_and_pharynx'] = \
diseaseCodesByGroup['malignant_neoplasms_of_digestive_organs'] + \
diseaseCodesByGroup['malig_neoplasm_of_lip_oral_cavity_and_pharynx']

targett = ['cerebrovascular_diseases', 'circulatory_diseases', 'diabetes_mellitus', 'digestive_diseases', 'diseases_of_liver',\
            'ischaemic_heart_diseases', 'malig_neoplasm_of_breast', 'malign_neoplasms_of_digestive_organs',\
           'malig_neoplasm_of_female_genital_organs', 'malig_neoplasm_of_lip_oral_cavity_and_pharynx',\
           'neoplasms_of_lymphoid_haematopoietic_and_related_tissue', 'malignant_neoplasm_of_male_genital_organs',\
           'malig_neoplasms_of_respiratory_and_intrathoracic_organs',\
           'melanoma_and_other_malignant_neoplasms_of_skin', 'neoplasms', 'malignant_neoplasms_of_urinary_tract']

## Skip this is only working with summary codes

In [23]:
from collections import defaultdict

more_detailed_codes = defaultdict(list)

for disease_group in diseaseCodesByGroup:
    for disease_code in diseaseCodesByGroup[disease_group]:
        for i in range(10):
            more_detailed_codes[disease_group].append(disease_code + str(i))
            
for disease_group in diseaseCodesByGroup:
    diseaseCodesByGroup[disease_group] += more_detailed_codes[disease_group]
    
diseaseCodesByGroup

{'certain_infectious_and_parasitic_diseases': ['A00',
  'A01',
  'A02',
  'A03',
  'A04',
  'A05',
  'A06',
  'A07',
  'A08',
  'A09',
  'A15',
  'A16',
  'A17',
  'A18',
  'A19',
  'A20',
  'A21',
  'A22',
  'A23',
  'A24',
  'A25',
  'A26',
  'A27',
  'A28',
  'A30',
  'A31',
  'A32',
  'A33',
  'A34',
  'A35',
  'A36',
  'A37',
  'A38',
  'A39',
  'A40',
  'A41',
  'A42',
  'A43',
  'A44',
  'A45',
  'A46',
  'A47',
  'A48',
  'A49',
  'A50',
  'A51',
  'A52',
  'A53',
  'A54',
  'A55',
  'A56',
  'A57',
  'A58',
  'A59',
  'A60',
  'A61',
  'A62',
  'A63',
  'A64',
  'A65',
  'A66',
  'A67',
  'A68',
  'A69',
  'A70',
  'A71',
  'A72',
  'A73',
  'A74',
  'A75',
  'A76',
  'A77',
  'A78',
  'A79',
  'A80',
  'A81',
  'A82',
  'A83',
  'A84',
  'A85',
  'A86',
  'A87',
  'A88',
  'A89',
  'A92',
  'A93',
  'A94',
  'A95',
  'A96',
  'A97',
  'A98',
  'A99',
  'B00',
  'B01',
  'B02',
  'B03',
  'B04',
  'B05',
  'B06',
  'B07',
  'B08',
  'B09',
  'B15',
  'B16',
  'B17',
  'B18',
 

In [43]:
causes = neoplasms

data = filter_and_merge(data_fao_with_codes, ['WHO code', 'Year'], data_icd, ['Country', 'Year'], causes)
data

Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               32.0                  
        1993                                               34.0                  
        1994                                               58.0                  
        1995                                               41.0                  
        1996                                               41.0                  
...                                                         ...                  
China   1996                                               69.0                  
        1997                                               72.0                  
        1998                                               61.0                  
        1999                                               58.0                  
        2000                                               58.0                  

             Animal Products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                 
Armenia 1992                                              355.0              
        1993                                              267.0              
        1994                                              372.0              
        1995                                              327.0              
        1996                                              312.0              
...                                                         ...              
China   1996                                              436.0              
        1997                                              450.0              
        1998                                              478.0              
        1999                                              495.0              
        2000                                              524.0              

             Animal fats,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                             
Armenia 1992                                               29.0          
        1993                                               56.0          
        1994                                               56.0          
        1995                                               73.0          
        1996                                               60.0          
...                                                         ...          
China   1996                                               30.0          
        1997                                               29.0          
        1998                                               31.0          
        1999                                               33.0          
        2000                                               39.0          

             Apples and products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               23.0                  
        1993                                                8.0                  
        1994                                               12.0                  
        1995                                               16.0                  
        1996                                               15.0                  
...                                                         ...                  
China   1996                                               12.0                  
        1997                                               12.0                  
        1998                                               13.0                  
        1999                                               14.0   

## HDI

In [76]:
from sklearn.linear_model import LinearRegression

hdi_data = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\HDI.csv")

hdi_data = hdi_data.drop(columns=['HDI Rank (2018)'])

columns_to_convert = list(hdi_data.columns)
columns_to_convert.remove("Country")

hdi_data[columns_to_convert] = hdi_data[columns_to_convert].apply(lambda x: pd.to_numeric(x, errors='coerce'))

hdi_data = hdi_data.melt(id_vars=["Country"], var_name="Year", value_name="HDI")

hdi_data = hdi_data.pivot_table(values='HDI', index=['Country', 'Year'], aggfunc='first')

hdi_data = hdi_data.reset_index()

hdi_data["Year"] = hdi_data["Year"].astype(int)

countries = hdi_data["Country"].unique()

for i, country in enumerate(countries):
    X = hdi_data.loc[hdi_data["Country"] == country]["Year"]
    y = hdi_data.loc[hdi_data["Country"] == country]["HDI"]
    X = X.to_numpy().reshape(-1, 1)
    reg = LinearRegression().fit(X, y)
    X_pred = [year for year in range(1990, X[0][0])]
    if not X_pred:
        continue
    X_pred = np.asarray(X_pred, dtype=np.int).reshape(-1, 1)
    y_pred = reg.predict(X_pred)
    n_predicted = len(y_pred)
    year_column = [val[0] for val in X_pred]
    hdi_column = list(y_pred)
    to_append = pd.DataFrame({"Country": [country] * n_predicted,\
                             "Year": year_column,\
                              "HDI": hdi_column})
    hdi_data = hdi_data.append(to_append)
        
#hdi_data.to_csv(r"C:\Users\Aleksandar\Downloads\Mortality\HDI_interpolated.csv")

hdi_data["Year"] = hdi_data["Year"].astype(int)

hdi_data

,Country,Year,HDI
0,Afghanistan,1990,0.298000
1,Afghanistan,1991,0.304000
2,Afghanistan,1992,0.312000
3,Afghanistan,1993,0.308000
4,Afghanistan,1994,0.303000
...,...,...,...
10,Vanuatu,2000,0.565007
11,Vanuatu,2001,0.566782
12,Vanuatu,2002,0.568558
13,Vanuatu,2003,0.570334


In [45]:
data = pd.merge(data.reset_index(), hdi_data.reset_index(), on=['Country', 'Year'])

data = data.set_index(['Country', 'Year'])

data

Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               32.0                  
        1993                                               34.0                  
        1994                                               58.0                  
        1995                                               41.0                  
        1996                                               41.0                  
...                                                         ...                  
China   1996                                               69.0                  
        1997                                               72.0                  
        1998                                               61.0                  
        1999                                               58.0                  
        2000                                               58.0                  

             Animal Products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                 
Armenia 1992                                              355.0              
        1993                                              267.0              
        1994                                              372.0              
        1995                                              327.0              
        1996                                              312.0              
...                                                         ...              
China   1996                                              436.0              
        1997                                              450.0              
        1998                                              478.0              
        1999                                              495.0              
        2000                                              524.0              

             Animal fats,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                             
Armenia 1992                                               29.0          
        1993                                               56.0          
        1994                                               56.0          
        1995                                               73.0          
        1996                                               60.0          
...                                                         ...          
China   1996                                               30.0          
        1997                                               29.0          
        1998                                               31.0          
        1999                                               33.0          
        2000                                               39.0          

             Apples and products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               23.0                  
        1993                                                8.0                  
        1994                                               12.0                  
        1995                                               16.0                  
        1996                                               15.0                  
...                                                         ...                  
China   1996                                               12.0                  
        1997                                               12.0                  
        1998                                               13.0                  
        1999                                               14.0   

## Population data

In [77]:
population_data = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\population_data.csv")
population_data = population_data.drop(columns=['Indicator Name', 'Indicator Code', 'Country Code'])
population_data = population_data.melt(id_vars=["Country Name"], var_name="Year", value_name="Population")
population_data = population_data.rename({'Country Name': 'Country'}, axis=1)
population_data['Country'] = population_data['Country'].astype(str)
population_data['Year'] = population_data['Year'].astype(int)
population_data = population_data.set_index(['Country', 'Year'])

population_data

,,Population
Country,Year,
Aruba,1960,54211.0
Afghanistan,1960,8996973.0
Angola,1960,5454933.0
Albania,1960,1608800.0
Andorra,1960,13411.0
...,...,...
Kosovo,2019,1794248.0
"Yemen, Rep.",2019,29161922.0
South Africa,2019,58558270.0


In [47]:
data = pd.merge(data.reset_index(), population_data.reset_index(), on=['Country', 'Year'])

data = data.set_index(['Country', 'Year'])

data

Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               32.0                  
        1993                                               34.0                  
        1994                                               58.0                  
        1995                                               41.0                  
        1996                                               41.0                  
...                                                         ...                  
China   1996                                               69.0                  
        1997                                               72.0                  
        1998                                               61.0                  
        1999                                               58.0                  
        2000                                               58.0                  

             Animal Products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                 
Armenia 1992                                              355.0              
        1993                                              267.0              
        1994                                              372.0              
        1995                                              327.0              
        1996                                              312.0              
...                                                         ...              
China   1996                                              436.0              
        1997                                              450.0              
        1998                                              478.0              
        1999                                              495.0              
        2000                                              524.0              

             Animal fats,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                             
Armenia 1992                                               29.0          
        1993                                               56.0          
        1994                                               56.0          
        1995                                               73.0          
        1996                                               60.0          
...                                                         ...          
China   1996                                               30.0          
        1997                                               29.0          
        1998                                               31.0          
        1999                                               33.0          
        2000                                               39.0          

             Apples and products,Food supply (kcal/capita/day),kcal/capita/day  \
Country Year                                                                     
Armenia 1992                                               23.0                  
        1993                                                8.0                  
        1994                                               12.0                  
        1995                                               16.0                  
        1996                                               15.0                  
...                                                         ...                  
China   1996                                               12.0                  
        1997                                               12.0                  
        1998                                               13.0                  
        1999                                               14.0   

## Average yearly temperature data

In [78]:
fields = ['Temperature - (Celsius)', 'Year', 'Country']

data_temp = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\temperature_data.csv", skipinitialspace=True, usecols=fields)

data_temp = data_temp.groupby(['Country', 'Year']).mean()

data_temp = data_temp.reset_index()

for country in data_temp["Country"].unique():
    mean_temp = data_temp.loc[data_temp["Country"] == country]['Temperature - (Celsius)'].mean()
    row_2017 = pd.DataFrame({'Country': [country], 'Year': [2017], 'Temperature - (Celsius)': [mean_temp]})
    data_temp = data_temp.append(row_2017)

data_temp = data_temp.set_index(["Country", "Year"])
data_temp = data_temp.sort_index()
data = pd.merge(data.reset_index(), data_temp.reset_index(), on=['Country', 'Year'])

data = data.set_index(['Country', 'Year'])

data_temp

Temperature - (Celsius)
Country     Year                         
Afghanistan 1991                12.894488
            1992                12.544548
            1993                12.977138
            1994                13.080018
            1995                13.040895
...                                   ...
Zimbabwe    2013                21.624350
            2014                21.710483
            2015                22.327625
            2016                22.310167
            2017                22.040161

[5265 rows x 1 columns]

## Percent of NaNs

In [79]:
def percent_nan(data):
    return data.isna().sum().sum() / (data.shape[0] * data.shape[1])

## Reduce the percent of NaN values to a given value
### Arguments: dataframe and threshold value, output is a dataframe with a percent of NaN values reduced to the threshold value

In [80]:
def reduce_nan(data, threshold):
    
    data_copy = data.reset_index()
    data_copy['Id'] = list(range(data_copy.shape[0]))
    
    while percent_nan(data_copy) > threshold:
        #append to the list tuples (a, b) where a is a string ROW__x where x is the row number,
        #b is the ratio of NaN values to all values in the row
        rows_and_cols = [("__".join(["ROW", str(row)]),\
                          data_copy[data_copy["Id"] == row].isna().sum().sum() / data_copy.shape[1])\
                         for row in data_copy["Id"].values]
        
        #append to the list tuples (a, b) where a is a string COL__x where x is the column name,
        #b is the ratio of NaN values to all values in the column
        rows_and_cols.extend([("__".join(["COL", col]),\
                               data_copy.loc[:, col].isna().sum() / data_copy.shape[0])\
                              for col in data_copy.columns])
       
        #sort by Nan to all values ratio, largest first (with most NaN values)
        rows_and_cols = sorted(rows_and_cols, reverse=True, key=lambda x: x[1])
        
        #lists of rows and columns to be removed
        cols_to_remove = []
        rows_to_remove = []
        
        #remove top 30 rows/columns
        for i in range(30):
            #take current top
            greatest = rows_and_cols[i][0]
            
            #divide into type (column or row) and identifier
            kind, name = greatest.split('__')
            
            #check if it is row or column and remove it
            if kind == "ROW":
                rows_to_remove.append(int(name))
            else:
                cols_to_remove.append(name)
        
        #remove elements
        data_copy = data_copy.drop(rows_to_remove, axis=0)
        data_copy = data_copy.drop(cols_to_remove, axis=1)
        print(f"Nans at {percent_nan(data_copy)} percent")
        
    data_copy = data_copy.drop('Id', axis=1)
        
    return data_copy

In [52]:
data_reduced = reduce_nan(data, 0.2)

data_reduced

,Country,Year,"Alcoholic Beverages,Food supply (kcal/capita/day),kcal/capita/day","Animal Products,Food supply (kcal/capita/day),kcal/capita/day","Animal fats,Food supply (kcal/capita/day),kcal/capita/day","Apples and products,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Animals, Others,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Plants,Food supply (kcal/capita/day),kcal/capita/day","Aquatic Products, Other,Food supply (kcal/capita/day),kcal/capita/day","Bananas,Food supply (kcal/capita/day),kcal/capita/day",...,"Vegetal Products,Food supply (kcal/capita/day),kcal/capita/day","Wheat and products,Food supply (kcal/capita/day),kcal/capita/day","Wine,Food supply (kcal/capita/day),kcal/capita/day","Yams,Food supply (kcal/capita/day),kcal/capita/day",Area Code,Total Deaths,index,HDI,Population,Temperature - (Celsius)
0,Armenia,1992,32.0,355.0,29.0,23.0,0.0,0.0,0.0,0.0,...,1829.0,1215.0,14.0,NaN,1,1258,200,0.585,3.442817e+06,5.805497
1,Armenia,1993,34.0,267.0,56.0,8.0,0.0,0.0,0.0,0.0,...,1832.0,1256.0,16.0,NaN,1,1208,201,0.590,3.363108e+06,6.107013
2,Armenia,1994,58.0,372.0,56.0,12.0,0.0,0.0,0.0,0.0,...,1809.0,1127.0,14.0,NaN,1,1104,202,0.600,3.283667e+06,7.173292
3,Armenia,1995,41.0,327.0,73.0,16.0,0.0,0.0,0.0,0.0,...,1943.0,1200.0,6.0,NaN,1,1130,203,0.604,3.217348e+06,7.867025
4,Armenia,1996,41.0,312.0,60.0,15.0,0.0,0.0,0.0,0.0,...,2008.0,1117.0,5.0,NaN,1,1244,204,0.614,3.168221e+06,7.914756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,China,1996,69.0,436.0,30.0,12.0,0.0,4.0,4.0,4.0,...,2267.0,680.0,1.0,0.0,351,150766,941,0.558,1.217550e+09,6.417824
1856,China,1997,72.0,450.0,29.0,12.0,0.0,3.0,4.0,4.0,...,2284.0,679.0,1.0,0.0,351,162326,942,0.566,1.230075e+09,6.843258
1857,China,1998,61.0,478.0,31.0,13.0,0.0,4.0,5.0,5.0,...,2298.0,676.0,2.0,0.0,351,164168,943,0.574,1.241935e+09,7.503825
1858,China,1999,58.0,495.0,33.0,14.0,0.0,5.0,5.0,6.0,...,2276.0,665.0,2.0,0.0,351,181176,944,0.583,1.252735e+09,7.299526


## Build datasets for each cause group

In [81]:
import pathlib

correlations = pd.read_excel(r"C:\Users\Aleksandar\Downloads\Mortality\output\Mortality and Food correlations.xlsx")

target_from_excel = correlations['Cause'].unique()
finished_from_excel = []

foods_for_cause = dict()

for cause in target_from_excel:
    foods_for_cause[cause] = list(correlations.loc[correlations['Cause'] == cause, 'Food'].unique())
    
output_folder_name = 'icd9 summary, scaled data, combined columns, lag'
pathlib.Path(r'C:\Users\Aleksandar\Downloads\Mortality\output\datasets\{}'.format(output_folder_name)).mkdir(parents=True, exist_ok=True) 
pathlib.Path(r'C:\Users\Aleksandar\Downloads\Mortality\output\results\{}'.format(output_folder_name)).mkdir(parents=True, exist_ok=True) 
pathlib.Path(r'C:\Users\Aleksandar\Downloads\Mortality\output\filtered_results\{}'.format(output_folder_name)).mkdir(parents=True, exist_ok=True)

all_foods_causes = list(set([(food, cause) for food, cause in correlations[['Food', 'Cause']].values]))
#for k, v in foods_for_cause.items():
    #print(k, v)

In [82]:
from linearmodels import RandomEffects, PanelOLS, PooledOLS
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.compat import lzip
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scale_data = True
include_hdi = True
include_temperature = False
reduce_data = True
interpolate_data = True
include_population = False
scale_foods = False

for cause, causes in diseaseCodesByGroup.items():
# for food, cause in all_foods_causes:
    if cause not in target_from_excel or cause in finished_from_excel:
        continue
    if cause in ['metabolic_syndrome']:
        continue
        
    print(f"CAUSE: {cause}")

    # only summary codes
    data_icd = data_icd.loc[~(data_icd['isSummary'] == False)]
    # filter and merge by causes
    data = filter_and_merge(data_fao_with_codes, ['WHO code', 'Year'], data_icd, ['Country', 'Year'], causes)
    
    # merge with HDI
    if include_hdi:
        data = pd.merge(data.reset_index(), hdi_data.reset_index(), on=['Country', 'Year'])
        data = data.set_index(['Country', 'Year'])
        
    # merge with population data
    if include_population:
        data = pd.merge(data.reset_index(), population_data.reset_index(), on=['Country', 'Year'])
        data['Total Deaths'] = (data['Total Deaths'] * 100) / data['Population']
        data = data.drop(columns='Population')
        data = data.set_index(['Country', 'Year'])
        
    # merge with temperature data
    if include_temperature:
        data = pd.merge(data.reset_index(), data_temp.reset_index(), on=['Country', 'Year'])
        data = data.set_index(['Country', 'Year'])
        
    # reduce percent of NaNs
    data_reduced = data.loc[:, :]
    if reduce_data:
        data_reduced = reduce_nan(data, 0.05)
        data_reduced = data_reduced.set_index(['Country', 'Year'])
        print("Reduced Nans", data_reduced.shape)
        
    # interpolate
    data_reduced = data_reduced.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    if interpolate_data:
        data_reduced = data_reduced.interpolate(method='linear', limit_direction='backward', axis=1)
        print("converted to numeric, interpolated")
        
        # scale food intake
    if scale_foods:
        columns = [col for col in data_reduced.columns if col not in ['Total Deaths', 'HDI', 'Temperature - (Celsius)', \
                          'Grand Total,Food supply (kcal/capita/day),kcal/capita/day', 'Area Code', 'index',]]
        for col in columns:
            data_reduced[col] = data_reduced[col] * 100 / data_reduced['Grand Total,Food supply (kcal/capita/day),kcal/capita/day']
    
    ########## Few Columns
#     if ';' in food:
#         food_cols = food[:].split(';')
#         food_cols = [col for col in food_cols if col in data_reduced.columns]
#         data_reduced[food] = data_reduced[food_cols].sum(axis=1)
#     data_reduced = data_reduced[[food, 'Total Deaths', 'Grand Total,Food supply (kcal/capita/day),kcal/capita/day', 'HDI']]
    ##########
    
    # drop columns with small number of different values
    nunique = data_reduced.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique < 5].index
    data_reduced = data_reduced.drop(cols_to_drop, axis=1)
    
    # create new columns and remove old ones
    cols_to_drop = []
    for foods in foods_for_cause[cause]:
        food_cols = foods.split(';')
        food_cols = [col for col in food_cols if col in data_reduced.columns]
        if len(food_cols) > 1:
            data_reduced[foods] = data_reduced[food_cols].sum(axis=1)
            cols_to_drop.extend([x for x in food_cols if x not in foods_for_cause[cause]])
    cols_to_drop = list(set(cols_to_drop))
    data_reduced = data_reduced.drop(cols_to_drop, axis=1)
    
    # scale
    data_scaled = data_reduced.copy()
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 100))
        columns = list(data_scaled.columns)
        columns.remove('Total Deaths')
        data_scaled = data_scaled.reset_index()
        data_scaled[columns] = scaler.fit_transform(data_scaled[columns])
#         data_scaled.loc[data_scaled['Total Deaths'] == 0, 'Total Deaths'] = 1
#         data_scaled['Total Deaths'] = np.log(data_scaled['Total Deaths'])
        data_scaled = data_scaled.set_index(['Country', 'Year'])
    
    
    ### LAG column
    data_shifted = pd.DataFrame()
    data_scaled = data_scaled.reset_index()
    for country in data_scaled['Country'].unique():
        data_country = data_scaled.loc[data_scaled['Country'] == country]
        data_country['Total Deaths'] = data_country['Total Deaths'].shift(-3)
        data_country['HDI'] = data_country['HDI'].shift(-3)
        data_country.drop(data_country.tail(3).index, inplace=True)
        data_shifted = data_shifted.append(data_country)
    data_scaled = data_shifted.set_index(['Country', 'Year'])
        
    cols_to_drop = ['Area Code', \
                    'index', \
                    'Offals, Edible,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Offals,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Miscellaneous,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Infant food,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Stimulants,Food supply (kcal/capita/day),kcal/capita/day',
                    'Maize Germ Oil,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Maize and products,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Oilcrops, Other,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Barley and products,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Fish, Liver Oil,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Aquatic Products, Other,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Beverages, Fermented,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Pulses, Other and products,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Rye and products,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Rape and Mustardseed,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Coconut Oil,Food supply (kcal/capita/day),kcal/capita/day', \
                    'Molluscs, Other,Food supply (kcal/capita/day),kcal/capita/day'
                    ]   
    cols_to_drop = [col for col in cols_to_drop if col in data_scaled.columns]
    data_scaled = data_scaled.drop(cols_to_drop, axis=1)
    
     ### Few columns
    filtered_cols = foods_for_cause[cause]
    filtered_cols.extend(['Total Deaths', 'HDI', 'Grand Total,Food supply (kcal/capita/day),kcal/capita/day'])
    data_scaled = data_scaled[filtered_cols]
    ###

    file_name = cause
    data_scaled.to_csv(r"C:\Users\Aleksandar\Downloads\Mortality\output\datasets\{}\{}.csv"\
                       .format(output_folder_name, file_name))
    
    finished_from_excel.append(cause)

CAUSE: neoplasms
Nans at 0.0190119544865332 percent
Reduced Nans (1441, 103)
converted to numeric, interpolated


C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

CAUSE: mental_behavioural_disorders
Nans at 0.017405294719906393 percent
Reduced Nans (1290, 103)
converted to numeric, interpolated


C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

CAUSE: nervous_system_diseases
Nans at 0.018595343518555452 percent
Reduced Nans (1419, 103)
converted to numeric, interpolated


...

C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

CAUSE: diabetes
Nans at 0.010610877223129134 percent
Reduced Nans (226, 100)
converted to numeric, interpolated


C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

CAUSE: malignant_neoplasms_of_digestive_organs_and_malig_neoplasm_of_lip_oral_cavity_and_pharynx
Nans at 0.018708701536501872 percent
Reduced Nans (1422, 103)
converted to numeric, interpolated


C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Aleksandar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

## Run R script here for Hausman test

In [83]:
file = open(r"C:\Users\Aleksandar\Downloads\Mortality\r scripts\preferred_effects.txt")
lines = file.readlines()
file.close()

# Run best models for each dataset

In [84]:
for cause, causes in diseaseCodesByGroup.items():
# for food, cause in all_foods_causes:
    if cause not in target_from_excel:
        continue
    if cause in ['diabetes', 'metabolic_syndrome', 'malig_neoplasm_of_breast', 'ischaemic_heart_diseases', \
                'malignant_neoplasms_of_digestive_organs', 'all_causes']:
        continue
        
    print(cause)
#     file_name = create_file_name(food, cause)
    file_name = cause
    data_scaled = pd.read_csv(r"C:\Users\Aleksandar\Downloads\Mortality\output\datasets\{}\{}.csv".format(output_folder_name, file_name))
    data_scaled = data_scaled.set_index(['Country', 'Year'])
    
    columns = [col for col in data_scaled]
    columns.remove("Total Deaths")
    exog = sm.add_constant(data_scaled[columns])
    target = data_scaled["Total Deaths"]
    
    models = []
    
    results = sm.OLS(target, exog).fit()
    #print(results.summary())
    name = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
    test = sms.het_breuschpagan(results.resid, results.model.exog)
    p_value = test[1]
    if p_value > 0.05:
        models.append(PooledOLS(target, exog))
        
    # F-statistic test
    # If the null hypothesis of the F-test is rejected, a fixed effect model is favored over OLS
    f_prob = float(str(results.summary2()).split()[36])
    if f_prob > 0.05:
        if PooledOLS(target, exog) not in models:
            models.append(PooledOLS(target, exog))
            
    preferred_effects = None
    for line in lines:
        if line.startswith(cause):
            preferred_effects = str(line.split('=')[1][:2])

    if preferred_effects is None:
        continue
        
    if preferred_effects == "FE":
        models.append(PanelOLS(target, exog, entity_effects=True))
    else:
        models.append(RandomEffects(target, exog))
        
    print("Running models")
    
    print(f"Diseases: {cause}")
    path = r'C:\Users\Aleksandar\Downloads\Mortality\output\results\{}\{}.txt'.format(output_folder_name, file_name)
    output_file = open(path, 'w')
    output_file.write('===============================================\n')
    output_file.write(f'{cause}')
    output_file.write('===============================================\n')
    for model in models:
        print(f"Fitting {str(model)[:20]}")
        mod_res = model.fit()
        output_file.write(f'{str(mod_res)}\n')
        
    output_file.close()
#     finished_from_excel.append(cause)

neoplasms


C:\Users\Aleksandar\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Running models
Diseases: neoplasms
Fitting RandomEffects 
Num e
mental_behavioural_disorders
Running models
Diseases: mental_behavioural_disorders
Fitting RandomEffects 
Num e
nervous_system_diseases
Running models
Diseases: nervous_system_diseases
Fitting PanelOLS 
Num exog: 
circulatory_diseases
Running models
Diseases: circulatory_diseases
Fitting PanelOLS 
Num exog: 
respiratory_diseases
Running models
Diseases: respiratory_diseases
Fitting RandomEffects 
Num e
digestive_diseases
Running models
Diseases: digestive_diseases
Fitting PanelOLS 
Num exog: 
malignant_neoplasms_of_urinary_tract
Running models
Diseases: malignant_neoplasms_of_urinary_tract
Fitting RandomEffects 
Num e
cerebrovascular_diseases
Running models
Diseases: cerebrovascular_diseases
Fitting PanelOLS 
Num exog: 
diseases_of_liver
malig_neoplasm_of_female_genital_organs
Running models
Diseases: malig_neoplasm_of_female_genital_organs
Fitting RandomEffects 
Num e
malig_neoplasm_of_lip_oral_cavity_and_pharynx
Running 

# Filter only variables with p-value < 0.05

In [85]:
import os.path

def filter_results(p_threshold):
    skip_diseases = ['eye_adnexa_diseases', 'ear_mastoid_process_diseases', 'injury_poisoning_external_causes',\
                'factors_influencing_health_status_contact_with_health_services']
    
    input_folder = r'C:\Users\Aleksandar\Downloads\Mortality\output\results\{}'.format(output_folder_name)
    
    include_tokens = ['R-squared:', 'Log-likelihood', 'F-statistic:', 'P-value', 'Distribution:', 'Summary', 'Avg', \
                     'Min', 'Max', 'F-statistic']
    
    for cause, causes in diseaseCodesByGroup.items():
#     for food, cause in all_foods_causes:
        if cause in skip_diseases:
            continue
        
        input_path = f'{input_folder}\\{cause}.txt'
#         file_name = create_file_name(food, cause)
#         input_path = f'{input_folder}\\{file_name}.txt'
        file_name = cause
        if not os.path.isfile(input_path):
            continue
            
        input_file = open(input_path, 'r')

        output_path = r'C:\Users\Aleksandar\Downloads\Mortality\output\filtered_results\{}\{}.txt'.format(output_folder_name, file_name)
        output_file = open(output_path, 'w')
        for line in input_file:
            tokens = line.split()
            for token in include_tokens:
                if token in tokens:
                    output_file.write(line)
            if len(tokens) >= 9:
                p_val = 0
                try:
                    p_val = float(tokens[-3])
                except ValueError:
                    print("Not a float")
                if p_val <= p_threshold:
                    output_file.write(line)

        output_file.close()
        input_file.close()

In [86]:
filter_results(0.05)

Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float
Not a float


# Find coefficient for cause and food

In [87]:
from os import path

def find_coefficient(food, cause):
    input_folder = r'C:\Users\Aleksandar\Downloads\Mortality\output\filtered_results\{}'.format(output_folder_name)
    input_path = f'{input_folder}\\{cause}.txt'
#     file_name = create_file_name(food, cause)
#     input_path = f'{input_folder}\\{file_name}.txt'
    
    if not path.isfile(input_path):
        return None, None
    
    input_file = open(input_path, 'r')
    
    coefficient = None
    p_val = None
    for line in input_file.readlines():
        if line.startswith(food):
            coefficient = float(line.split()[-6])
            p_val = float(line.split()[-3])
        ###
#         all_foods_included = True
#         for fd in file_name.split('=')[0].split(';'):
#             if fd not in line:
#                 all_foods_included = False
#         if all_foods_included:
#             coefficient = float(line.split()[-6])
#             p_val = float(line.split()[-3])
        ###
    input_file.close()

    return coefficient, p_val

## Write obtained coefficients to excel

In [88]:
correlations = pd.read_excel(r"C:\Users\Aleksandar\Downloads\Mortality\output\Mortality and Food correlations.xlsx")

for idx, row in correlations.iterrows():
    food = row['Food']
    cause = row['Cause']
    coefficient, p_val = find_coefficient(food, cause)
    if coefficient is None:
        # p value > 0.05
        coefficient = -9000
        
    correlations.at[idx, 'Obtained Coefficient'] = coefficient
    correlations.at[idx, 'p-value'] = p_val
    
correlations.to_excel(r"C:\Users\Aleksandar\Downloads\Mortality\output\Mortality and Food correlations {}.xlsx".format(output_folder_name))

## Compare coefficients

In [89]:
def sign(expression):
    x = float(expression)
    if x == 0:
        return '0'
    return '+' if x > 0 else '-'

def n_matches():
    paper_correlations = [x for x in correlations['Correlation'].values]
    obtained_correlations = [x for x in correlations['Obtained Coefficient'].values]
    n = len(paper_correlations)
    match, dont_match = 0, 0
    for i in range(n):
        if obtained_correlations[i] != float(-9000):
            if sign(paper_correlations[i]) == sign(obtained_correlations[i]):
                match += 1
            else:
                dont_match += 1

    return match, dont_match

n_matches()

(8, 7)

In [90]:
with open(r"C:\Users\Aleksandar\Downloads\Mortality\output\matches.txt", "a") as file:
    file.write(f"\n{n_matches()} -> {output_folder_name}")